In [1]:
import json
import glob
import os
import numpy as np
import pandas as pd
pd.options.display.max_rows = 4
pd.options.display.max_columns = 8

In [2]:
results_cardio = [json.load(open(f)) for f in glob.glob('../cardio_results/*.json')]

In [3]:
def flatten(value):
    result = value.copy()
    result.update(result['bbox'])
    del result['bbox']
    return result

keys = ['quality', 'model_config',]
c_df = pd.DataFrame([flatten(r) for r in results_cardio]).set_index(keys).sort_index()
c_df

elapsed  \
quality model_config                                                
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml   2826.581140   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml  1645.898561   
...                                                           ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml    1301.285856   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml     1070.493649   

                                                                 finished  \
quality model_config                                                        
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml   2020-06-08 00:41:11   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml  2020-06-08 08:53:43   
...                                                                   ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml    2020-06-09 20:11:30   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml     2020-06-10 06:55:16   

                                                                    device  \
quality model_config                                                         
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml   GeForce GTX 1060 6GB   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml  GeForce GTX 1060 6GB   
...                                                                    ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml    GeForce GTX 1060 6GB   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml     GeForce GTX 1060 6GB   

                                                             AP  ...  \
quality model_config                                             ...   
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml    0.689457  ...   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml   0.740405  ...   
...                                                         ...  ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml    39.869614  ...   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml     37.924545  ...   

                                                      AP-scissors  \
quality model_config                                                
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml      0.000000   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml     0.000000   
...                                                           ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml      30.317981   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml       25.936524   

                                                      AP-teddy bear  \
quality model_config                                                  
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml        0.000000   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml       0.000000   
...                                                             ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml        44.312238   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml         42.228038   

                                                      AP-hair drier  \
quality model_config                                                  
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml        0.000000   
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml       0.000000   
...                                                             ...   
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml         0.595652   
        COCO-Detection/retinanet_R_50_FPN_3x.yaml          0.950829   

                                                      AP-toothbrush  
quality model_config                                                 
1       COCO-Detection/faster_rcnn_R_101_C4_3x.yaml        0.000000  
        COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml       0.000000  
...                                                             ...  
100     COCO-Detection/retinanet_R_101_FPN_3x.yaml        15.846204  
        COCO-Detection/retinanet_R_50_FPN_3x.yaml         14.429351  

[900 rows x

In [4]:
c_df.to_csv('../data/results_val2017.csv', float_format='%.3f')

## Merging means of quality and compression ratio

In [5]:
q_df = pd.read_csv('../data/val2017_means_per_Q.csv')
q_df

,quality,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize,B_per_kpx
0,1,536.180,21.205,0.539,7203.782,9.291
1,2,535.910,21.207,0.539,7207.225,9.295
...,...,...,...,...,...,...
98,99,0.224,54.850,0.999,215370.463,273.443
99,100,0.159,56.273,0.999,257475.181,325.305


In [6]:
df = pd.merge(c_df, q_df, on='quality')
# no float truncation: 1,4 MB; +%.3f,-device: 584 KB; -elapsed,-finished: 560 KB
remove = ['device', 'elapsed', 'finished', ]
df = df.drop(remove, 1)
df

,quality,AP,AP50,AP75,...,peak_signal_noise_ratio,structural_similarity,filesize,B_per_kpx
0,1,0.689457,1.235953,0.663127,...,21.205,0.539,7203.782,9.291
1,1,0.740405,1.303380,0.735289,...,21.205,0.539,7203.782,9.291
...,...,...,...,...,...,...,...,...,...
898,100,39.869614,60.102201,42.733239,...,56.273,0.999,257475.181,325.305
899,100,37.924545,57.871832,40.367879,...,56.273,0.999,257475.181,325.305


In [7]:
df.to_csv('../data/results_and_quality_val2017.csv', float_format='%.3f')